In [ ]:
!kill -KILL 300065

In [4]:
!nvidia-smi

Fri Jun 14 16:15:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:01:00.0 Off |                  Off |
| 31%   34C    P8              21W / 450W |      1MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
from inference import run_inference

ImportError: libcudnn.so.8: cannot open shared object file: No such file or directory

In [ ]:
run_inference(
    run_name="/cluster/work/lawecon/Work/mbraasch/output/qwen2-1.5-instruct/gsm8k-dl/baseline",
    postfix="_test",
    eos_token="<|im_end|>"
    #previous_next_steps="/cluster/work/lawecon/Work/mbraasch/output/gemma-2b-it/gsm8k-dl/m123-m4f-tf/m123/s1_next_step_predictions_s1_test.json",
    #data_dir = "/cluster/work/lawecon/Work/mbraasch/data/gsm8k_train.json"
)